# Data Preparation

## Overview

The purpose of this notebook is to handle the data preparation and merging of multiple datasets related to the Brazilian e-commerce data. The objective is to clean, standardize, and consolidate the data into a single, structured dataset for further analysis and modeling.

## Objectives

* Import datasets and verify their structure.
* Perform data quality checks to identify missing and duplicate values.
* Clean and preprocess each dataset individually.
* Standardize column names and data types to maintain consistency.
* Merge datasets to form a single, consolidated dataframe for analysis.
* Export the cleaned and merged dataset for further analysis.

## Dataset Structure

* **orders:** Order information, including purchase timestamps and order statuses.
* **customers:** Customer data, including unique identifiers and geolocation.
* **order\_items:** Product details and associated order information.
* **products:** Product metadata, including category and dimensions.
* **sellers:** Seller information, including location and unique IDs.
* **order\_reviews:** Customer reviews and ratings.
* **geolocation:** Geolocation data for delivery addresses.

## **Setup and Libraries:**

   * Import necessary libraries and define dataset paths.

In [2]:
import pandas as pd
import numpy as np
import sqlite3

## **Data Import:**

   * Load each dataset as a pandas DataFrame.

In [3]:
customers = pd.read_csv('data/olist_customers_dataset.csv')
geolocation = pd.read_csv('data/olist_geolocation_dataset.csv')
orders = pd.read_csv('data/olist_orders_dataset.csv')
order_items = pd.read_csv('data/olist_order_items_dataset.csv')
order_payments = pd.read_csv('data/olist_order_payments_dataset.csv')
order_reviews = pd.read_csv('data/olist_order_reviews_dataset.csv')
products = pd.read_csv('data/olist_products_dataset.csv')
sellers = pd.read_csv('data/olist_sellers_dataset.csv')
category_name_translation = pd.read_csv('data/product_category_name_translation.csv')

## **Data Quality Check:**

   * Analyze missing and duplicate values in each dataset.

In [4]:
# Summarize key characteristics of each dataset in a structured table.
# This approach ensures concise output and facilitates data quality checks.

# Define a dictionary of datasets for streamlined iteration
datasets = {
    'customers': customers,
    'geolocation': geolocation,
    'orders': orders,
    'order_items': order_items,
    'order_payments': order_payments,
    'order_reviews': order_reviews,
    'products': products,
    'sellers': sellers,
    'category_translation': category_name_translation  # Shortened for clarity
}

# Create a list to store summary statistics for each dataset
dataset_summaries = []

# Iterate through datasets to collect relevant metrics
for dataset_name, dataframe in datasets.items():
    # Calculate total missing values across all columns
    missing_values = dataframe.isnull().sum().sum()
    # Identify duplicate rows
    duplicates = dataframe.duplicated().sum()
    # Extract row and column counts
    row_count, column_count = dataframe.shape
    
    # Store metrics in a dictionary
    dataset_summaries.append({
        'Dataset': dataset_name.capitalize(),  # Format for readability
        'Missing Values': missing_values,
        'Duplicate Rows': duplicates,
        'Row Count': row_count,
        'Column Count': column_count
    })

# Convert the summaries to a DataFrame for structured output
summary_table = pd.DataFrame(dataset_summaries)

# Display the dataset summary table
summary_table.head(8)

,Dataset,Missing Values,Duplicate Rows,Row Count,Column Count
0,Customers,0,0,99441,5
1,Geolocation,0,261831,1000163,5
2,Orders,4908,0,99441,8
3,Order_items,0,0,112650,7
4,Order_payments,0,0,103886,5
5,Order_reviews,145903,0,99224,7
6,Products,2448,0,32951,9
7,Sellers,0,0,3095,4


### Chekcing the Results
* Now that we have a summary table showing the missing values and duplicated rows for each dataset, we can identify which datasets have the most significant data quality issues.
* With this information, we can focus our cleaning and preprocessing efforts on the datasets that present the highest number of missing or duplicated values, ensuring a more robust and reliable analysis in the next steps.

## **Data Cleaning:**

   * Remove duplicates and handle missing values.

#### Orders Dataset

In [5]:
# Check percentage of missing values in the orders dataset
orders.isnull().sum() / orders.shape[0] * 100

#### Note ####
## Considering the percentage of missing values in the orders dataset, 
## we can see that the columns 'order_approved_at ', 'order_delivered_customer_date' and 'order_delivered_customer_date' have a insignificant amount of missing data.
## We can drop these as they are not critical for our analysis.

order_id                         0.000000
customer_id                      0.000000
order_status                     0.000000
order_purchase_timestamp         0.000000
order_approved_at                0.160899
order_delivered_carrier_date     1.793023
order_delivered_customer_date    2.981668
order_estimated_delivery_date    0.000000
dtype: float64

In [6]:
# Drop the null rvalues from the orders dataset
orders.dropna(inplace=True)
# Check the percentage of missing values in the orders dataset again
orders.isnull().sum() / orders.shape[0] * 100

order_id                         0.0
customer_id                      0.0
order_status                     0.0
order_purchase_timestamp         0.0
order_approved_at                0.0
order_delivered_carrier_date     0.0
order_delivered_customer_date    0.0
order_estimated_delivery_date    0.0
dtype: float64

#### Orders Reviews Dataset

In [7]:
# Check the percentage of missing values in the order reviews dataset
order_reviews.isnull().sum() / order_reviews.shape[0] * 100

#### Note ####
## For the order reviews dataset, we can see that the columns 'review_comment_title' and 'review_answer_timestamp' 
## have a significant amount of missing data, but considreing the focus of our analysis will not be on the reviews comments, 
## I've decided to drop these columns

review_id                   0.000000
order_id                    0.000000
review_score                0.000000
review_comment_title       88.341530
review_comment_message     58.702532
review_creation_date        0.000000
review_answer_timestamp     0.000000
dtype: float64

In [8]:
# Drop the null columns from the order reviews dataset
order_reviews = order_reviews.drop(columns=['review_comment_title', 
                                             'review_comment_message'])
# Check the percentage of missing values in the order reviews dataset again
order_reviews.isnull().sum() / order_reviews.shape[0] * 100

review_id                  0.0
order_id                   0.0
review_score               0.0
review_creation_date       0.0
review_answer_timestamp    0.0
dtype: float64

#### Products Dataset

In [9]:
# Check the percentage of missing values in the products dataset
products.isnull().sum() / products.shape[0] * 100

#### Note ####
## For the products dataset, we can see that the columns 'product_description_lenght', 'product_photos_qty' and 'product_weight_g'
## have a insignificant amount of missing data.
## So I'll drop only the missing values rows from the products dataset


product_id                    0.000000
product_category_name         1.851234
product_name_lenght           1.851234
product_description_lenght    1.851234
product_photos_qty            1.851234
product_weight_g              0.006070
product_length_cm             0.006070
product_height_cm             0.006070
product_width_cm              0.006070
dtype: float64

In [10]:
# Drop the rows with missing values from the products dataset 
products.dropna(inplace=True)
# Check the percentage of missing values in the products dataset again
products.isnull().sum() / products.shape[0] * 100

product_id                    0.0
product_category_name         0.0
product_name_lenght           0.0
product_description_lenght    0.0
product_photos_qty            0.0
product_weight_g              0.0
product_length_cm             0.0
product_height_cm             0.0
product_width_cm              0.0
dtype: float64

In [11]:
category_name_translation.head()

,product_category_name,product_category_name_english
0,beleza_saude,health_beauty
1,informatica_acessorios,computers_accessories
2,automotivo,auto
3,cama_mesa_banho,bed_bath_table
4,moveis_decoracao,furniture_decor


In [12]:
# As we can see, the colum product_category_name is in portuguese,
# To change it to english, we can use the category_name_translation dataset
# Merge the products dataset with the category_name_translation dataset
products = products.merge(category_name_translation, 
                           how='left', 
                           on='product_category_name')

In [13]:
# Display the first 5 rows of the merged products dataset
# Compare the columns product_category_name and product_category_name_english are matching

products[['product_category_name', 'product_category_name_english']]

,product_category_name,product_category_name_english
0,perfumaria,perfumery
1,artes,art
2,esporte_lazer,sports_leisure
3,bebes,baby
4,utilidades_domesticas,housewares
...,...,...
32335,moveis_decoracao,furniture_decor
32336,construcao_ferramentas_iluminacao,construction_tools_lights
32337,cama_mesa_banho,bed_bath_table
32338,informatica_acessorios,computers_accessories


In [14]:
# Check the percentage of missing values again to ensure the merge was successful
products.isnull().sum() / products.shape[0] * 100

product_id                       0.000000
product_category_name            0.000000
product_name_lenght              0.000000
product_description_lenght       0.000000
product_photos_qty               0.000000
product_weight_g                 0.000000
product_length_cm                0.000000
product_height_cm                0.000000
product_width_cm                 0.000000
product_category_name_english    0.040198
dtype: float64

#### Geolocation Dataset
* The Geolocation dataset has two problemas:
    1. I identified duplicate values and then dropped those rows.
    2. Since each zipcode has a different latitude and longitude, using each combination would result in high memory usage and duplicate rows in the main dataset.
    
* To solve this, the SQL query will retrieve the average of each latitude and longitude. This query will be created later. 

In [15]:
# Drop duplicate rows from the geolocation dataset
geolocation.drop_duplicates(inplace=True)

In [16]:
# Check the duplicate rows in the geolocation dataset
geolocation.duplicated().sum()

np.int64(0)

In [17]:
# Display the dataset
geolocation.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
2,1046,-23.546129,-46.642951,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP


In [18]:
geolocation.shape

(738332, 5)

## **Data Standardization and Formatting:**

   * Standardize column names and adjust data types for consistency.

In [17]:
# Summarize all the datasets columns names and types
# Create a list to store column names and types for each dataset
dataset_columns = []
# Iterate through datasets to collect column names and types
for dataset_name, dataframe in datasets.items():
    # Get the column names and types
    columns = dataframe.dtypes.reset_index()
    columns.columns = ['Column Name', 'Data Type']
    # Add the dataset name to the columns DataFrame
    columns['Dataset'] = dataset_name.capitalize()
    # Append the columns DataFrame to the list
    dataset_columns.append(columns)
# Concatenate all the columns DataFrames into a single DataFrame
columns_summary = pd.concat(dataset_columns, ignore_index=True)
# Display the columns summary DataFrame
columns_summary.head(100)



,Column Name,Data Type,Dataset
0,customer_id,object,Customers
1,customer_unique_id,object,Customers
2,customer_zip_code_prefix,int64,Customers
3,customer_city,object,Customers
4,customer_state,object,Customers
5,geolocation_zip_code_prefix,int64,Geolocation
6,geolocation_lat,float64,Geolocation
7,geolocation_lng,float64,Geolocation
8,geolocation_city,object,Geolocation
9,geolocation_state,object,Geolocation


In [18]:
# Display all rows from columns_summary where the column name ends with '_id'
columns_summary[columns_summary['Column Name'].str.endswith('_id')]

,Column Name,Data Type,Dataset
0,customer_id,object,Customers
1,customer_unique_id,object,Customers
10,order_id,object,Orders
11,customer_id,object,Orders
18,order_id,object,Order_items
19,order_item_id,int64,Order_items
20,product_id,object,Order_items
21,seller_id,object,Order_items
25,order_id,object,Order_payments
30,review_id,object,Order_reviews


#### Conclusion
* There is no need of column name padronization
* Checked that the primary keys and foreign keys are string objects for the merge

*This is important cause keeping primary and foreign keys as strings ensures data integrity, prevents issues with leading zeros or mixed formats, and guarantees that joins between tables work correctly. IDs are identifiers, not values for mathematical operations, so treating them as strings avoids unintended data loss or mismatches during merges.*

## **Database and Merged Dataset:**
   * Create the database and tables, and then populate them.
   * Merge datasets based on common keys to form a comprehensive dataset.

### Database Tables Creation

In [ ]:
# Create a SQLite database
conn = sqlite3.connect("ecommerce.db")
cursor = conn.cursor()

# Create the tables with relationships and constraints
cursor.execute("""
CREATE TABLE customers (
    customer_id TEXT PRIMARY KEY,
    customer_unique_id TEXT NOT NULL,
    customer_zip_code_prefix INTEGER NOT NULL
);
""")

cursor.execute("""
CREATE TABLE geolocation (
    zip_code_prefix INTEGER PRIMARY KEY,
    geolocation_lat REAL NOT NULL,
    geolocation_lng REAL NOT NULL
);
""")

cursor.execute("""
CREATE TABLE orders (
    order_id TEXT PRIMARY KEY,
    customer_id TEXT NOT NULL,
    order_status TEXT NOT NULL,
    order_purchase_timestamp TEXT NOT NULL,
    order_approved_at TEXT,
    order_delivered_carrier_date TEXT,
    order_delivered_customer_date TEXT,
    order_estimated_delivery_date TEXT,
    FOREIGN KEY (customer_id) REFERENCES customers (customer_id)
);
""")

cursor.execute("""
CREATE TABLE order_items (
    order_id TEXT NOT NULL,
    product_id TEXT NOT NULL,
    seller_id TEXT NOT NULL,
    PRIMARY KEY (order_id, product_id, seller_id),
    FOREIGN KEY (order_id) REFERENCES orders (order_id),
    FOREIGN KEY (product_id) REFERENCES products (product_id),
    FOREIGN KEY (seller_id) REFERENCES sellers (seller_id)
);
""")

cursor.execute("""
CREATE TABLE products (
    product_id TEXT PRIMARY KEY,
    product_category_name TEXT,
    product_category_name_english TEXT
);
""")

cursor.execute("""
CREATE TABLE sellers (
    seller_id TEXT PRIMARY KEY,
    seller_zip_code_prefix INTEGER NOT NULL,
    FOREIGN KEY (seller_zip_code_prefix) REFERENCES geolocation (zip_code_prefix)
);
""")

cursor.execute("""
CREATE TABLE order_payments (
    order_id TEXT NOT NULL,
    payment_type TEXT NOT NULL,
    payment_installments INTEGER NOT NULL,
    payment_value REAL NOT NULL,
    FOREIGN KEY (order_id) REFERENCES orders (order_id)
);
""")

cursor.execute("""
CREATE TABLE order_reviews (
    order_id TEXT NOT NULL,
    review_score INTEGER NOT NULL,
    review_creation_date TEXT NOT NULL,
    review_answer_timestamp TEXT NOT NULL,
    PRIMARY KEY (order_id),
    FOREIGN KEY (order_id) REFERENCES orders (order_id)
);
""")

# Commit the changes and close the connection
conn.commit()
print("Database and tables created successfully with relationships and constraints.")
conn.close()

Database and tables created successfully with relationships and constraints.


### Database Population

In [ ]:
# Create a connection to the SQLite database
conn = sqlite3.connect("ecommerce.db")  # Use the same database file created earlier

# Populate the database tables with the DataFrames
customers.to_sql("customers", conn, index=False, if_exists="replace")
geolocation.to_sql("geolocation", conn, index=False, if_exists="replace")
orders.to_sql("orders", conn, index=False, if_exists="replace")
order_items.to_sql("order_items", conn, index=False, if_exists="replace")
order_payments.to_sql("order_payments", conn, index=False, if_exists="replace")
order_reviews.to_sql("order_reviews", conn, index=False, if_exists="replace")
products.to_sql("products", conn, index=False, if_exists="replace")
sellers.to_sql("sellers", conn, index=False, if_exists="replace")

# Confirm the data has been inserted
print("Database tables populated successfully.")

# Close the connection
conn.close()

Database tables populated successfully.


### Merging Datasets with SQLite Query

As previously mentioned, to efficiently merge data and save memory, the SQL query will calculate the average `latitude` and `longitude` from the `Geolocation` dataset.


In [39]:
import os
# Define the database path (modify this to your actual database file path)
db_path = "ecommerce.db"

# Define the output CSV path
output_dir = "data/processed_dataset"
output_path = os.path.join(output_dir, "merged_data.csv")

# SQL query from your previous request
sql_query = """
SELECT 
    o.order_id,
    o.customer_id,
    o.order_status,
    o.order_purchase_timestamp,
    o.order_approved_at,
    o.order_delivered_carrier_date,
    o.order_delivered_customer_date,
    o.order_estimated_delivery_date,
    c.customer_unique_id,
    c.customer_zip_code_prefix,
    c.customer_city,
    c.customer_state,
    cg.geolocation_lat AS customer_lat,
    cg.geolocation_lng AS customer_lng,
    oi.order_item_id,
    oi.product_id,
    oi.seller_id,
    oi.price,
    oi.freight_value,
    oi.shipping_limit_date,
    p.product_category_name_english AS product_category_name,
    p.product_name_lenght,
    p.product_description_lenght,
    p.product_photos_qty,
    p.product_weight_g,
    p.product_length_cm,
    p.product_height_cm,
    p.product_width_cm,
    s.seller_zip_code_prefix,
    s.seller_city,
    s.seller_state,
    sg.geolocation_lat AS seller_lat,
    sg.geolocation_lng AS seller_lng,
    op.payment_sequential,
    op.payment_type,
    op.payment_installments,
    op.payment_value,
    r.review_id,
    r.review_score,
    r.review_creation_date,
    r.review_answer_timestamp
FROM 
    orders o
LEFT JOIN 
    customers c ON o.customer_id = c.customer_id
LEFT JOIN (
    SELECT 
        geolocation_zip_code_prefix,
        AVG(geolocation_lat) AS geolocation_lat,
        AVG(geolocation_lng) AS geolocation_lng
    FROM 
        geolocation
    GROUP BY 
        geolocation_zip_code_prefix
) cg ON c.customer_zip_code_prefix = cg.geolocation_zip_code_prefix
LEFT JOIN 
    order_items oi ON o.order_id = oi.order_id
LEFT JOIN 
    products p ON oi.product_id = p.product_id
LEFT JOIN 
    sellers s ON oi.seller_id = s.seller_id
LEFT JOIN (
    SELECT 
        geolocation_zip_code_prefix,
        AVG(geolocation_lat) AS geolocation_lat,
        AVG(geolocation_lng) AS geolocation_lng
    FROM 
        geolocation
    GROUP BY 
        geolocation_zip_code_prefix
) sg ON s.seller_zip_code_prefix = sg.geolocation_zip_code_prefix
LEFT JOIN 
    order_payments op ON o.order_id = op.order_id
LEFT JOIN 
    order_reviews r ON o.order_id = r.order_id;
"""

try:
    # Connect to the SQLite database
    conn = sqlite3.connect(db_path)
    print(f"Connected to database: {db_path}")

    # Execute the query and load results into a pandas DataFrame
    df = pd.read_sql_query(sql_query, conn)
    print(f"Query executed successfully. Retrieved {len(df)} rows.")

    # Close the database connection
    conn.close()
    print("Database connection closed.")

   
    # Save the DataFrame to a CSV file
    df.to_csv(output_path, index=False, encoding='utf-8')
    print(f"Data saved to CSV: {output_path}")

except sqlite3.Error as e:
    print(f"SQLite error: {e}")
except Exception as e:
    print(f"An error occurred: {e}")

Connected to database: ecommerce.db
Query executed successfully. Retrieved 115706 rows.
Database connection closed.
Data saved to CSV: data/processed_dataset\merged_data.csv


### Merged Data

In [40]:
merged_data = pd.read_csv('data/processed_dataset/merged_data.csv')
merged_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 115706 entries, 0 to 115705
Data columns (total 41 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       115706 non-null  object 
 1   customer_id                    115706 non-null  object 
 2   order_status                   115706 non-null  object 
 3   order_purchase_timestamp       115706 non-null  object 
 4   order_approved_at              115706 non-null  object 
 5   order_delivered_carrier_date   115706 non-null  object 
 6   order_delivered_customer_date  115706 non-null  object 
 7   order_estimated_delivery_date  115706 non-null  object 
 8   customer_unique_id             115706 non-null  object 
 9   customer_zip_code_prefix       115706 non-null  int64  
 10  customer_city                  115706 non-null  object 
 11  customer_state                 115706 non-null  object 
 12  customer_lat                  

In [41]:
merged_data.shape

(115706, 41)

In [42]:
merged_data.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'customer_unique_id', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'customer_lat', 'customer_lng', 'order_item_id',
       'product_id', 'seller_id', 'price', 'freight_value',
       'shipping_limit_date', 'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'seller_zip_code_prefix', 'seller_city', 'seller_state', 'seller_lat',
       'seller_lng', 'payment_sequential', 'payment_type',
       'payment_installments', 'payment_value', 'review_id', 'review_score',
       'review_creation_date', 'review_answer_timestamp'],
      dtype='object')

### Merged Data Quality Check

In [45]:
# Check the duplicate rows in the merged dataset
merged_data.duplicated().sum()

np.int64(0)

In [46]:
# Determine the missing value percentage in the merged dataset, 
# filtering to include only columns with null entries.
null_percent = merged_data.isnull().sum() / merged_data.shape[0] * 100
null_percent[null_percent > 0]

customer_lat                  0.261871
customer_lng                  0.261871
product_category_name         1.435535
product_name_lenght           1.415657
product_description_lenght    1.415657
product_photos_qty            1.415657
product_weight_g              1.415657
product_length_cm             1.415657
product_height_cm             1.415657
product_width_cm              1.415657
seller_lat                    0.225572
seller_lng                    0.225572
payment_sequential            0.002593
payment_type                  0.002593
payment_installments          0.002593
payment_value                 0.002593
review_id                     0.744127
review_score                  0.744127
review_creation_date          0.744127
review_answer_timestamp       0.744127
dtype: float64

In [47]:
# Drop the missing values from the merged dataset
merged_data.dropna(inplace=True)
merged_data.isnull().sum() / merged_data.shape[0] * 100

order_id                         0.0
customer_id                      0.0
order_status                     0.0
order_purchase_timestamp         0.0
order_approved_at                0.0
order_delivered_carrier_date     0.0
order_delivered_customer_date    0.0
order_estimated_delivery_date    0.0
customer_unique_id               0.0
customer_zip_code_prefix         0.0
customer_city                    0.0
customer_state                   0.0
customer_lat                     0.0
customer_lng                     0.0
order_item_id                    0.0
product_id                       0.0
seller_id                        0.0
price                            0.0
freight_value                    0.0
shipping_limit_date              0.0
product_category_name            0.0
product_name_lenght              0.0
product_description_lenght       0.0
product_photos_qty               0.0
product_weight_g                 0.0
product_length_cm                0.0
product_height_cm                0.0
p

## **Export Clean Data:**

   * Save the cleaned and consolidated dataset for further analysis.

In [48]:
# Save the merged dataset to the specified directory
output_path = "data/processed_dataset/merged_data_clean.csv"
merged_data.to_csv(output_path, index=False)

print(f"Merged data saved to '{output_path}'.")

Merged data saved to 'data/processed_dataset/merged_data_clean.csv'.


## Output

* The merged dataset will be saved as `merged_data.csv` in the `processed` data folder.

## Next Steps

* Proceed to the next notebook: `02_analysis_and_modeling.ipynb` for exploratory data analysis and machine learning modeling.